In [6]:
import torch
import torch.nn as nn
import numpy as np
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [9]:
# prepare data

bc = datasets.load_breast_cancer()
X, y = bc.data, bc.target

n_samples, n_features = X.shape
print(n_samples, n_features)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

# scale features
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

X_train = torch.from_numpy(X_train.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32))
X_test = torch.from_numpy(X_test.astype(np.float32))
y_test = torch.from_numpy(y_test.astype(np.float32))

# reshape into column vector
y_train = y_train.view(y_train.shape[0], 1)
y_test = y_test.view(y_test.shape[0], 1)

569 30


In [10]:
# model
# f = wx+b, sigmoid at the end

class LogisticRegression(nn.Module):
    
    def __init__(self, n_input_features):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(n_input_features, 1)
        
    def forward(self, x):
        y_pred = torch.sigmoid(self.linear(x))
        return y_pred
    
model = LogisticRegression(n_features)

In [11]:
# loss and optimizer

learning_rate = 0.01

criterion = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [20]:
# training loop

n_epochs = 100
for epoch in range(n_epochs):
    # forward pass
    y_pred = model(X_train)
    
    loss = criterion(y_pred, y_train)
    
    # backward pass
    loss.backward()
    
    # update weights
    optimizer.step()
    
    optimizer.zero_grad()
    
    if epoch+1 % 10 == 0:
        print(f'epoch {epoch+1}: loss = {loss.item():.4f}')
    
# evaluation with no grad calculations
with torch.no_grad():
    y_pred = model(X_test)
    y_pred_classes = y_pred.round() # rounds if pred is >= 0.5, don't want to calc gradient on round
    acc = y_pred_classes.eq(y_test).sum()/float(y_test.shape[0])
    print(f'accuracy = {acc:.4f}')

accuracy = 0.9298
